In [1]:
import random

In [2]:
random.seed(42)

In [3]:
p10 = []
p8 = []
ep = []
ip = []
ip_inv = [1]*8

In [4]:
ip=  [1, 8, 3, 6, 5, 4, 7, 2]
ip_inv=  [7, 8, 5, 1, 2, 6, 3, 4]
ep=  [4, 3, 2, 1, 4, 3, 2, 1]
p10=  [4, 6, 8, 5, 9, 7, 5, 3, 2, 1]
p8=  [1, 2, 3, 5, 7, 9, 4, 6]

In [5]:
for i in range(len(ip)):
    ip_inv[ip[i]-1]=i+1
ip_inv

[1, 8, 3, 6, 5, 4, 7, 2]

In [6]:
# def generate_random_permutation(n):
#     out = [i+1 for i in range(n)]
#     random.shuffle(out)
#     return out

In [7]:
# ip = generate_random_permutation(8)
# ip_inv = [1]*8
# for i in range(len(ip)):
#     ip_inv[ip[i]-1] = i+1

In [8]:
# print('IP: ', ip)
# print('IP_INV: ', ip_inv)

In [9]:
# p10 = generate_random_permutation(10)
# print('P10: ', p10)

In [10]:
# ep = generate_random_permutation(4)*2
# print(ep)

In [11]:
# p8 = random.sample(generate_random_permutation(10), 8)
# print('P8: ', p8)

In [12]:
def bin_to_dec(x):
    return int(x, 2)
def dec_to_bin(x):
    return bin(x).replace('0b', '')

In [13]:
def left_circular_shift(x, shifts=1):
    shifts = shifts % len(x)
    return x[shifts:] + x[:shifts]

In [14]:
def permutate(key, perm):
    ret = []
    for k in perm:
        ret.append(str(key[k-1]))
    return ''.join(ret)

In [15]:
def split_str(key):
    mid = len(key)//2
    return key[:mid], key[mid:]

In [16]:
def xor(a, b):
    ret = []
    for i in range(len(a)):
        if a[i] == b[i]: ret.append('0')
        else: ret.append('1')
    return ''.join(ret)

In [17]:
def generate_subkeys(key):
    new_key = permutate(key,p10)
    
    left_key, right_key = split_str(new_key)
    
    # generate k1
    left_key = left_circular_shift(left_key, 1)
    right_key = left_circular_shift(right_key, 1)
    
    k1 = permutate(left_key+right_key, p8)
    
    # generate k2
    left_key = left_circular_shift(left_key, 2)
    right_key = left_circular_shift(right_key, 2)
    
    k2 = permutate(left_key+right_key, p8)
    
    return k1, k2

In [18]:
key = "1010001011"
k1, k2 = generate_subkeys(key)
print(k1)
print(k2)

00001110
01001000


In [19]:
s0 = [
    [1, 0, 3, 2],
    [3, 2, 1, 0],
    [0, 2, 1, 3],
    [3, 1, 3, 2]
]
s1 = [
    [0, 1, 2, 3],
    [2, 0, 1, 3],
    [3, 0, 1, 0],
    [2, 1, 0, 3]
]

In [20]:
def s_box(text, sbox):
    r = text[0] + text[3]
    c = text[1] + text[2]
    
    r = bin_to_dec(r)
    c = bin_to_dec(c)
    
    out = sbox[r][c]

    out = dec_to_bin(out)

    while len(out) < 2:
        out = '0' + out
    return out

In [61]:
def function(left, right, subkey):
    text = right
    text = permutate(text, ep)
    text = xor(text, subkey)
    text_left, text_right = split_str(text)
    text = s_box(text_left, s0) + s_box(text_right, s1)
    text = xor(text, left)
    return text, right

In [53]:
def function2(left, right, subkey):
    text = right
    text = permutate(text, ep)
    text = xor(text, subkey)
    text_left, text_right = split_str(text)
    text = s_box(texleft, s0) + s_box(right, s1)
    text = xor(text, left)
    return text, right

In [54]:
def encryption(plaintext, key):
    k1, k2 = generate_subkeys(key)
    
    ciphertext = permutate(plaintext, ip)
    print('permutated ciphertext: ', ciphertext)
    left, right = split_str(ciphertext)
    left, right = function(left, right, k1)
    print('left, right: ', left, right)
    left, right = right, left
    
    left, right = function(left, right, k2)
    print('left, right: ', left, right)
    ciphertext = permutate(left+right, ip_inv)
    
    return ciphertext

In [55]:
def decryption(ciphertext, key):
    k1, k2 = generate_subkeys(key)
    
    plaintext = permutate(ciphertext, ip)
    
    left, right = split_str(plaintext)
    left, right = function(left, right, k2)
    
    left, right = right, left
    
    left, right = function(left, right, k1)
    
    plaintext = permutate(left+right, ip_inv)
    
    return plaintext

In [56]:
key = "1010001011"
plaintext = "10001010"

In [62]:
c = encryption(plaintext, key)

permutated ciphertext:  10001010
left, right:  1101 1010
left, right:  0010 1101


In [63]:
p = decryption(c, key)

In [64]:
plaintext

'10001010'

In [65]:
p

'10001010'